In [30]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if ".avi" in file:
            files.append(os.path.join(r, file))
        
for f in files:
    print(f)
np.array(files).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\nonsleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\sleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\slowBlinkWithNodding.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\yawning.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\nonsleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\sleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\slowBlinkWithNodding.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\yawning.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\nonsleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\sleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\slow

(20,)

In [31]:
Statelist=['nonsleepyCombination','sleepyCombination','slowBlinkWithNodding','yawning']
for i in range(4):
    print(files[i][55:58])
    print(files[i][67:-4])
    print(files[i][54:-4])
    print('----')

001
nonsleepyCombination
\001\glasses\nonsleepyCombination
----
001
sleepyCombination
\001\glasses\sleepyCombination
----
001
slowBlinkWithNodding
\001\glasses\slowBlinkWithNodding
----
001
yawning
\001\glasses\yawning
----


## MTCNN

if face not detected in 60 frames, use the last detected 

In [32]:
from mtcnn.mtcnn import MTCNN
# import the necessary packages
from imutils import face_utils
import dlib
import cv2
import numpy as np
import os
from operator import itemgetter
import time

eyes_width = 90
eyes_height = 50
mouth_width = 75
mouth_height = 50
face_width = 105
face_height = 125
num_classes = 2
channel = 1



def CreateFolder(name):
    try:
        # creating a folder named data 
        if not os.path.exists(name):
            os.makedirs(name)
        # if not created then raise error 
    except OSError: 
        print ('Error: Creating directory of data')            
#==============================================================

detector = MTCNN()

for filename in files:
    
    #==========================
    f = open('droplist.txt','a')
    f.write(filename+"\n")
    f.close()
    #==========================
    # img = cv2.imread("me.jpg")
    
    #filename = '.\\Dataset\\Training_Evaluation_Dataset\\Training Dataset\\001\\glasses\\yawning.avi'

    print(filename)
    cap = cv2.VideoCapture(filename)
    #================================================================ create new folder

    DefaultPath = '.\\Dataset\\Training_Evaluation_Dataset\\Training Image\\'+filename[54:-4]+"\\"
    CreateFolder(DefaultPath+"face")
    CreateFolder(DefaultPath+"eyes")
    CreateFolder(DefaultPath+"mouth")
    #================================================================
    Face_ImageDataList = []

    currentframe = 0
    missing_frame = 0
    DropList = []

    eyes_w = eyes_width
    eyes_h = eyes_height
    face_w = face_width
    face_h = face_height
    mouth_w = mouth_width
    mouth_h = mouth_height
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(length)
    x,y,w,h = 0,0,0,0
    
    while(cap.isOpened()):
        
        currentframe += 1
        # load the input image and convert it to grayscale
        ret, image = cap.read()
        if (not ret):
          break
        
        start_time = time.time()
        

        
        #3 frame sample once
        if(currentframe%3!=0):
            continue
        
        #if(currentframe<2200):
        #    continue
        

        height,width = image.shape[:2]
        #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #image = cv2.resize(image,(width//2,height//2),interpolation=cv2.INTER_CUBIC)
        #print(image.shape[:2])
        faces = detector.detect_faces(image)
        #print(np.array(faces).shape[0])
        
        if(np.array(faces).shape[0] > 0):
            face = faces[0]
            x,y,w,h = face['box']
            
            face_image = image[max(0,y):max(0,y+h),max(0,x):max(0,x+w)]
            #print(face_image.shape)
            #print(face_image.shape)
            #print(image[x:x+w,y:y+h])
            #cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            face_image = cv2.resize(face_image,(face_w,face_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face\\frame" + str(currentframe) + '.jpg',face_image) 


            eyes_image = image[max(0,face['keypoints']['left_eye'][1]-int(h*0.2)):max(0,face['keypoints']['right_eye'][1]+int(h*0.2))
                                ,max(0,face['keypoints']['left_eye'][0]-int(w*0.2)):max(0,face['keypoints']['right_eye'][0]+int(w*0.2))]
            
            #cv2.rectangle(image, (face['keypoints']['left_eye'][0]-int(w*0.2), face['keypoints']['left_eye'][1]-int(h*0.2)),
            #              (face['keypoints']['right_eye'][0]+int(w*0.2), face['keypoints']['right_eye'][1]+int(h*0.2)),(0, 255, 0),2)
            eyes_image = cv2.resize(eyes_image,(eyes_w,eyes_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes\\frame" + str(currentframe) + '.jpg',eyes_image) 


            mouth_image = image[max(0,face['keypoints']['mouth_left'][1]-int(h*0.2)):max(0,face['keypoints']['mouth_right'][1]+int(h*0.2))
                               ,max(0,face['keypoints']['mouth_left'][0]-int(w*0.2)):max(0,face['keypoints']['mouth_right'][0]+int(w*0.2))]
            #cv2.rectangle(image, (face['keypoints']['mouth_left'][0]-int(w*0.2), face['keypoints']['mouth_left'][1]-int(h*0.2)),
            #              (face['keypoints']['mouth_right'][0]+int(w*0.2), face['keypoints']['mouth_right'][1]+int(h*0.2)),(0, 255, 0),2)
            mouth_image = cv2.resize(mouth_image,(mouth_w,mouth_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth\\frame" + str(currentframe) + '.jpg',mouth_image) 



            #cv2.rectangle(image, (face['keypoints']['left_eye'][0], face['keypoints']['left_eye'][1]), 2, (0, 255, 0), -1)


            #cv2.circle(image, (face['keypoints']['left_eye'][0], face['keypoints']['left_eye'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['right_eye'][0], face['keypoints']['right_eye'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['nose'][0], face['keypoints']['nose'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['mouth_left'][0], face['keypoints']['mouth_left'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['mouth_right'][0], face['keypoints']['mouth_right'][1]), 2, (0, 255, 0), -1)
            if(currentframe%100==0):        
                name = './data/frame' + str(currentframe) + '.jpg'
                print ('Creating...' + name) 
        elif (missing_frame<60 and (not((x == 0) or (y == 0) or (w == 0) or (h == 0) ))):
            print ('missing using last frame') 
            missing_frame = missing_frame + 1        

            face_image = image[max(0,y):max(0,y+h),max(0,x):max(0,x+w)]
            print(y)
            print(y+h)
            print(x)
            print(x+w)
            #print(image[x:x+w,y:y+h])
            #cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            face_image = cv2.resize(face_image,(face_w,face_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face\\frame" + str(currentframe) + '.jpg',face_image) 


            eyes_image = image[max(0,face['keypoints']['left_eye'][1]-int(h*0.2)):max(0,face['keypoints']['right_eye'][1]+int(h*0.2))
                                ,max(0,face['keypoints']['left_eye'][0]-int(w*0.2)):max(0,face['keypoints']['right_eye'][0]+int(w*0.2))]
            #cv2.rectangle(image, (face['keypoints']['left_eye'][0]-int(w*0.2), face['keypoints']['left_eye'][1]-int(h*0.2)),
            #              (face['keypoints']['right_eye'][0]+int(w*0.2), face['keypoints']['right_eye'][1]+int(h*0.2)),(0, 255, 0),2)
            eyes_image = cv2.resize(eyes_image,(eyes_w,eyes_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes\\frame" + str(currentframe) + '.jpg',eyes_image) 


            mouth_image = image[max(0,face['keypoints']['mouth_left'][1]-int(h*0.2)):max(0,face['keypoints']['mouth_right'][1]+int(h*0.2))
                               ,max(0,face['keypoints']['mouth_left'][0]-int(w*0.2)):max(0,face['keypoints']['mouth_right'][0]+int(w*0.2))]
            #cv2.rectangle(image, (face['keypoints']['mouth_left'][0]-int(w*0.2), face['keypoints']['mouth_left'][1]-int(h*0.2)),
            #              (face['keypoints']['mouth_right'][0]+int(w*0.2), face['keypoints']['mouth_right'][1]+int(h*0.2)),(0, 255, 0),2)
            mouth_image = cv2.resize(mouth_image,(mouth_w,mouth_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth\\frame" + str(currentframe) + '.jpg',mouth_image) 
            
            #if(currentframe%100==0):
            name = './data/frame' + str(currentframe) + '.jpg'
            print ('missing using last frame:  '+ 'Creating...' + name) 

        else:
            name = './data/frame' + str(currentframe) + '.jpg'
            f = open('droplist.txt','a')
            f.write(name+"\n")
            f.close()
            DropList.append(currentframe)
            print("drop")



        '''
        cv2.imshow('img',image)

        #print("FPS: ", 1.0 / (time.time() - start_time))

        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
        '''
    
    print("total_frame:" +str(length)+" missing frame"+ str(np.array(DropList).shape[0])+" total frame2:"+ str(currentframe))
    print(DropList)
    cap.release()
    cv2.destroyAllWindows()

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\nonsleepyCombination.avi
3330
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1800.jpg
Creating..../data/frame2100.jpg
Creating..../data/frame2400.jpg
Creating..../data/frame2700.jpg
Creating..../data/frame3000.jpg
Creating..../data/frame3300.jpg
total_frame:3330 missing frame0 total frame2:3331
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\sleepyCombination.avi
2748
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1800.jpg
Creating..../data/frame2100.jpg
Creating..../data/frame2400.jpg
Creating..../data/frame2700.jpg
total_frame:2748 missing frame0 total frame2:2749
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\sl

missing using last frame
156
267
73
159
missing using last frame:  Creating..../data/frame2484.jpg
missing using last frame
156
267
73
159
missing using last frame:  Creating..../data/frame2487.jpg
Creating..../data/frame2700.jpg
total_frame:2732 missing frame0 total frame2:2733
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\noglasses\sleepyCombination.avi
2711
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1800.jpg
Creating..../data/frame2100.jpg
Creating..../data/frame2400.jpg
Creating..../data/frame2700.jpg
total_frame:2711 missing frame0 total frame2:2712
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\noglasses\slowBlinkWithNodding.avi
1852
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1800

### TODO
1. 所有file 存檔案
2. training 進CNN model
3. 存沒有東西的frame

### define CNN [model](https://medium.com/@mannasiladittya/building-inception-resnet-v2-in-keras-from-scratch-a3546c4d93f0)


In [1]:
%run ./Model.ipynb 

C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [3]:
img_input = Input(shape=(face_height,face_width,1))

x = conv2d(img_input,32,3,2,'valid',True,name='conv1')
x = conv2d(x,32,3,1,'valid',True,name='conv2')
x = conv2d(x,64,3,1,'valid',True,name='conv3')

x_11 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
x_12 = conv2d(x,96,3,1,'valid',True,name='stem_br_12')

x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

x_21 = conv2d(x,64,1,1,'same',True,name='stem_br_211')
x_21 = conv2d(x_21,64,[1,7],1,'same',True,name='stem_br_212')
x_21 = conv2d(x_21,64,[7,1],1,'same',True,name='stem_br_213')
x_21 = conv2d(x_21,96,3,1,'valid',True,name='stem_br_214')

x_22 = conv2d(x,64,1,1,'same',True,name='stem_br_221')
x_22 = conv2d(x_22,96,3,1,'valid',True,name='stem_br_222')

x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

x_31 = conv2d(x,192,3,1,'valid',True,name='stem_br_31')
x_32 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])

In [4]:
#Inception-ResNet-A modules
x = incresA(x,0.15,name='incresA_1')
x = incresA(x,0.15,name='incresA_2')
#x = incresA(x,0.15,name='incresA_3')
#x = incresA(x,0.15,name='incresA_4')

#35 × 35 to 17 × 17 reduction module.
x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)

x_red_12 = conv2d(x,384,3,2,'valid',True,name='x_red1_c1')

x_red_13 = conv2d(x,256,1,1,'same',True,name='x_red1_c2_1')
x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])

#Inception-ResNet-B modules
x = incresB(x,0.1,name='incresB_1')
x = incresB(x,0.1,name='incresB_2')
x = incresB(x,0.1,name='incresB_3')
#x = incresB(x,0.1,name='incresB_4')
#x = incresB(x,0.1,name='incresB_5')
#x = incresB(x,0.1,name='incresB_6')
#x = incresB(x,0.1,name='incresB_7')

#17 × 17 to 8 × 8 reduction module.
x_red_21 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_2')(x)

x_red_22 = conv2d(x,256,1,1,'same',True,name='x_red2_c11')
x_red_22 = conv2d(x_red_22,384,3,2,'valid',True,name='x_red2_c12')

x_red_23 = conv2d(x,256,1,1,'same',True,name='x_red2_c21')
x_red_23 = conv2d(x_red_23,256,3,2,'valid',True,name='x_red2_c22')

x_red_24 = conv2d(x,256,1,1,'same',True,name='x_red2_c31')
x_red_24 = conv2d(x_red_24,256,3,1,'same',True,name='x_red2_c32')
x_red_24 = conv2d(x_red_24,256,3,2,'valid',True,name='x_red2_c33')

x = Concatenate(axis=3, name='red_concat_2')([x_red_21,x_red_22,x_red_23,x_red_24])

#Inception-ResNet-C modules
x = incresC(x,0.2,name='incresC_1')
x = incresC(x,0.2,name='incresC_2')
#x = incresC(x,0.2,name='incresC_3')

#TOP
x = GlobalAveragePooling2D(data_format='channels_last')(x)
x = Dropout(0.6)(x)
#x = Dense(num_classes, activation='softmax')(x)
flat3 = Flatten()(x)

In [5]:



# first input model: eyes
eyes1 = Input(shape=(eyes_height,eyes_width,1))
conv11 = Conv2D(32, kernel_size=(3, 3), activation='relu')(eyes1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool11)
#pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
#flat1 = Flatten()(pool12)
pool12 = GlobalAveragePooling2D(data_format='channels_last')(conv12)
flat1 = Flatten()(pool12)


# second input model: mouth
mouth1 = Input(shape=(mouth_height,mouth_width,1))
conv21 = Conv2D(32, kernel_size=(3, 3), activation='relu')(mouth1)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=(3, 3), activation='relu')(pool21)
#pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
#flat2 = Flatten()(pool22)
pool22 = GlobalAveragePooling2D(data_format='channels_last')(conv22)
flat2 = Flatten()(pool22)

merge = Concatenate(axis=1, name='concat_2')([flat1, flat2,flat3])
output = Dense(num_classes, activation='softmax')(merge)

In [6]:
model = Model(inputs=[eyes1, mouth1, img_input], outputs=output, name='inception_resnet_v2')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 50, 1)    0                                            
__________________________________________________________________________________________________
conv1conv2d (Conv2D)            (None, 31, 24, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
conv1conv2dbn (BatchNormalizati (None, 31, 24, 32)   96          conv1conv2d[0][0]                
__________________________________________________________________________________________________
conv1conv2dact (Activation)     (None, 31, 24, 32)   0           conv1conv2dbn[0][0]              
__________________________________________________________________________________________________
conv2conv2

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
'''
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

SVG(model_to_dot(model).create(prog='dot', format='svg'))
plot_model(model, to_file='model_plot.png', 
                  show_shapes=True, show_layer_names=True)
'''

"\nfrom keras.utils.vis_utils import model_to_dot\nfrom tensorflow.keras.utils import plot_model\nfrom IPython.display import SVG\n\nSVG(model_to_dot(model).create(prog='dot', format='svg'))\nplot_model(model, to_file='model_plot.png', \n                  show_shapes=True, show_layer_names=True)\n"

In [9]:

# first input model: eyes
eyes1 = Input(shape=(eyes_height,eyes_width,1))
conv11 = Conv2D(32, kernel_size=(3, 3), activation='relu')(eyes1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool11)
#pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
#flat1 = Flatten()(pool12)
pool12 = GlobalAveragePooling2D(data_format='channels_last')(conv12)
flat1 = Flatten()(pool12)


# second input model: mouth
mouth1 = Input(shape=(mouth_height,mouth_width,1))
conv21 = Conv2D(32, kernel_size=(3, 3), activation='relu')(mouth1)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=(3, 3), activation='relu')(pool21)
#pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
#flat2 = Flatten()(pool22)
pool22 = GlobalAveragePooling2D(data_format='channels_last')(conv22)
flat2 = Flatten()(pool22)

# input model: face
face1 = Input(shape=(face_height,face_width,1))

x = incresA(face1,0.15,name='incresA_1')

#35 × 35 to 17 × 17 reduction module.
x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)

x_red_12 = conv2d(x,384,3,2,'valid',True,name='x_red1_c1')

x_red_13 = conv2d(x,256,1,1,'same',True,name='x_red1_c2_1')
x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])

x = incresB(x,0.1,name='incresB_1')

x = GlobalAveragePooling2D(data_format='channels_last')(x)
x = Dropout(0.6)(x)
flat3 = Flatten()(x)

merge = Concatenate(axis=1, name='concat_2')([flat1, flat2,flat3])
output = Dense(num_classes, activation='softmax')(merge)

In [10]:
model2 = Model(inputs=[eyes1, mouth1, face1], outputs=output, name='inception_resnet_v2')
model2.summary()
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 50, 1)    0                                            
__________________________________________________________________________________________________
incresA_1b2_1conv2d (Conv2D)    (None, 64, 50, 32)   32          input_6[0][0]                    
__________________________________________________________________________________________________
incresA_1b2_1conv2dbn (BatchNor (None, 64, 50, 32)   96          incresA_1b2_1conv2d[0][0]        
__________________________________________________________________________________________________
incresA_1b2_1conv2dact (Activat (None, 64, 50, 32)   0           incresA_1b2_1conv2dbn[0][0]      
__________________________________________________________________________________________________
incresA_1b

### Load one file and label

In [1]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

AllLabelFile = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "drowsiness.txt" in file :
            AllLabelFile.append(os.path.join(r, file))
        
for f in AllLabelFile:
    print(f)
np.array(AllLabelFile).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_sleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_slowBlinkWithNodding_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_yawning_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_sleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_slowBlinkWithNodding_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_yawning_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Tra

(356,)

In [2]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("nonsleepyCombination" in directory or "sleepyCombination" in directory or "slowBlinkWithNodding"  in directory or "yawning" in directory)   :
            #print(directory)
            AllImageFolder.append(os.path.join(r, directory))
        
for d in AllImageFolder:
    print(d)
np.array(AllImageFolder).shape

.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Trainin

(356,)

In [ ]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2

def LoadImageandLabel(Originalpath,labelPath):
    print(Originalpath)
    Face_ImageDataList = []
    Eyes_ImageDataList = []
    Mouth_ImageDataList = []
    #=====load all label in a video
    LabelList = []
    TempLabelList = []
    inputfile = open(labelPath, 'r').read()
    TempLabelList = list(map(int, inputfile))
    
    #=========================
    
    judgeflag = 0
    
    for i in range(3):
        TEMP_ImageDataList = []
        if(i==0):
            path = Originalpath+"\\eyes"
        elif(i==1):
            path = Originalpath+"\\face"
        else:
            path = Originalpath+"\\mouth"
            
        #=====find all files in path=======    
        files = []
        # r=root, d=directories, f = files
        for r, d, f in os.walk(path):
            for file in f:
                if ".jpg" in file:
                    files.append(os.path.join(r, file))


        files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
        for f in files:
            #======================load label
            if(i==0):
                index = f.find('frame')
                framenumber =int(f[index+5:-4]) 
                #print(framenumber)
                if(judgeflag == 0 and TempLabelList[framenumber-1] == 1):
                    judgeflag = 1
                    print(framenumber-1)
                    
                    
                LabelList.append(TempLabelList[framenumber-1])
                
                #Labelarray = np.array(LabelList)
                #Labelarray = to_categorical(Labelarray, num_classes=2)
            
            #================================
            ImgGray = cv2.imread(f,0)
            #ImgRGB = cv2.merge((ImgGray,ImgGray,ImgGray))# this line make gray image to RGB
            #TEMP_ImageDataList.append(ImgRGB)  

            ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
            #print(ImgGray.shape)
            TEMP_ImageDataList.append(ImgGray)
            #print(np.array(Face_ImageDataList).shape)
         #====================================
            
        
        if(i==0):
            Eyes_ImageDataList = TEMP_ImageDataList
            #print(LabelList)
        elif(i==1):
            Face_ImageDataList = TEMP_ImageDataList
        else:
            Mouth_ImageDataList = TEMP_ImageDataList
            
        
    return Face_ImageDataList , Mouth_ImageDataList , Eyes_ImageDataList




for epoch in range(50):
    index = 0
    for Video in AllImageFolder:
        
        #if(index == 0 ):
        #    index = index + 1
        #    continue
        print("---")
        #=========================load file
        
        faceImage = []
        mouthImage = []
        eyesImage = []

        labelfile = AllLabelFile[index]
        index = index + 1
        #print("label: "+labelfile)
        #==================================
        
        faceImage,mouthImage,eyesImage = LoadImageandLabel(Video,labelfile)
        
        #print(eyesImage)
        '''
        print(np.array(eyesImage).shape)
        print(np.array(faceImage).shape)
        print(np.array(mouthImage).shape)
        print("---")
        print("---")
        print("---")
        '''
        #plt.imshow(faceImage[1] , cmap = 'gray')
        
        #model_result = model2.fit([eyesImage,mouthImage,faceImage], Labelarray,batch_size=20,epochs=epoch+1, initial_epoch=epoch, shuffle=True,verbose = 1)



C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


---
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
(1110, 25, 45, 1)
(1110, 64, 50, 1)
(1110, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
599
(916, 25, 45, 1)
(916, 64, 50, 1)
(916, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
464
(640, 25, 45, 1)
(640, 64, 50, 1)
(640, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
341
(616, 25, 45, 1)
(616, 64, 50, 1)
(616, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
(507, 25, 45, 1)
(507, 64, 50, 1)
(507, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
242
(500, 25, 45, 1)
(500, 64, 50, 1)
(500, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasse

(1028, 25, 45, 1)
(1028, 64, 50, 1)
(1028, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\005\sunglasses\sleepyCombination
2
(1005, 25, 45, 1)
(1005, 64, 50, 1)
(1005, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\005\sunglasses\slowBlinkWithNodding
299
(763, 25, 45, 1)
(763, 64, 50, 1)
(763, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\005\sunglasses\yawning
242
(768, 25, 45, 1)
(768, 64, 50, 1)
(768, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\006\glasses\nonsleepyCombination
(956, 25, 45, 1)
(956, 64, 50, 1)
(956, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\006\glasses\sleepyCombination
128
(949, 25, 45, 1)
(949, 64, 50, 1)
(949, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\006\glasses\slowBlinkWithNodding
317
(637, 25, 45, 1)
(637, 64, 50, 1)
(637, 25, 35, 1)
---
---
---
---
.

(325, 25, 45, 1)
(325, 64, 50, 1)
(325, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\009\night_noglasses\yawning
233
(325, 25, 45, 1)
(325, 64, 50, 1)
(325, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\009\noglasses\nonsleepyCombination
(956, 25, 45, 1)
(956, 64, 50, 1)
(956, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\009\noglasses\sleepyCombination
422
(961, 25, 45, 1)
(961, 64, 50, 1)
(961, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\009\noglasses\slowBlinkWithNodding
440
(666, 25, 45, 1)
(666, 64, 50, 1)
(666, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\009\noglasses\yawning
341
(652, 25, 45, 1)
(652, 64, 50, 1)
(652, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\009\sunglasses\nonsleepyCombination
(980, 25, 45, 1)
(980, 64, 50, 1)
(980, 25, 35, 1)
---
---
---
---
.\Dataset

(766, 25, 45, 1)
(766, 64, 50, 1)
(766, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\015\nightglasses\nonsleepyCombination
(418, 25, 45, 1)
(418, 64, 50, 1)
(418, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\015\nightglasses\sleepyCombination
131
(473, 25, 45, 1)
(473, 64, 50, 1)
(473, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\015\nightglasses\slowBlinkWithNodding
152
(346, 25, 45, 1)
(346, 64, 50, 1)
(346, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\015\nightglasses\yawning
158
(325, 25, 45, 1)
(325, 64, 50, 1)
(325, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\015\night_noglasses\nonsleepyCombination
(420, 25, 45, 1)
(420, 64, 50, 1)
(420, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\015\night_noglasses\sleepyCombination
200
(478, 25, 45, 1)
(478, 64, 50, 1)
(478, 25, 35, 1

...

(767, 25, 45, 1)
(767, 64, 50, 1)
(767, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\035\noglasses\yawning
212
(705, 25, 45, 1)
(705, 64, 50, 1)
(705, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\035\sunglasses\nonsleepyCombination
(960, 25, 45, 1)
(960, 64, 50, 1)
(960, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\035\sunglasses\sleepyCombination
200
(988, 25, 45, 1)
(988, 64, 50, 1)
(988, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\035\sunglasses\slowBlinkWithNodding
260
(746, 25, 45, 1)
(746, 64, 50, 1)
(746, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\035\sunglasses\yawning
194
(701, 25, 45, 1)
(701, 64, 50, 1)
(701, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\036\glasses\nonsleepyCombination
(1055, 25, 45, 1)
(1055, 64, 50, 1)
(1055, 25, 35, 1)
---
---
---
---
.\Dataset\T

(304, 25, 45, 1)
(304, 64, 50, 1)
(304, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\002\night_noglasses\nonsleepyCombination
(420, 25, 45, 1)
(420, 64, 50, 1)
(420, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\002\night_noglasses\sleepyCombination
92
(460, 25, 45, 1)
(460, 64, 50, 1)
(460, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\002\night_noglasses\slowBlinkWithNodding
47
(307, 25, 45, 1)
(307, 64, 50, 1)
(307, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\002\night_noglasses\yawning
101
(304, 25, 45, 1)
(304, 64, 50, 1)
(304, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\002\noglasses\nonsleepyCombination
(815, 25, 45, 1)
(815, 64, 50, 1)
(815, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\002\noglasses\sleepyCombination
239
(885, 25, 45, 1)
(885, 64, 50, 1)
(885, 25, 35, 1)


(475, 64, 50, 1)
(475, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\008\nightglasses\sleepyCombination
2
(416, 25, 45, 1)
(416, 64, 50, 1)
(416, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\008\nightglasses\slowBlinkWithNodding
245
(318, 25, 45, 1)
(318, 64, 50, 1)
(318, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\008\nightglasses\yawning
200
(323, 25, 45, 1)
(323, 64, 50, 1)
(323, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\008\night_noglasses\nonsleepyCombination
(470, 25, 45, 1)
(470, 64, 50, 1)
(470, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\008\night_noglasses\sleepyCombination
203
(476, 25, 45, 1)
(476, 64, 50, 1)
(476, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\008\night_noglasses\slowBlinkWithNodding
176
(320, 25, 45, 1)
(320, 64, 50, 1)
(320, 25, 35, 1)
---
---
--

(972, 25, 45, 1)
(972, 64, 50, 1)
(972, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\012\sunglasses\slowBlinkWithNodding
380
(731, 25, 45, 1)
(731, 64, 50, 1)
(731, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\012\sunglasses\yawning
431
(725, 25, 45, 1)
(725, 64, 50, 1)
(725, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\013\glasses\nonsleepyCombination
230
(1077, 25, 45, 1)
(1077, 64, 50, 1)
(1077, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\013\glasses\sleepyCombination
230
(962, 25, 45, 1)
(962, 64, 50, 1)
(962, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\013\glasses\slowBlinkWithNodding
416
(748, 25, 45, 1)
(748, 64, 50, 1)
(748, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\013\glasses\yawning
350
(705, 25, 45, 1)
(705, 64, 50, 1)
(705, 25, 35, 1)
---
---
---
---
.\Dataset\T

(319, 25, 45, 1)
(319, 64, 50, 1)
(319, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\020\noglasses\nonsleepyCombination
(976, 25, 45, 1)
(976, 64, 50, 1)
(976, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\020\noglasses\sleepyCombination
275
(965, 25, 45, 1)
(965, 64, 50, 1)
(965, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\020\noglasses\slowBlinkWithNodding
251
(867, 25, 45, 1)
(867, 64, 50, 1)
(867, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\020\noglasses\yawning
221
(694, 25, 45, 1)
(694, 64, 50, 1)
(694, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\020\sunglasses\nonsleepyCombination
(974, 25, 45, 1)
(974, 64, 50, 1)
(974, 25, 35, 1)
---
---
---
---
.\Dataset\Training_Evaluation_Dataset\Training Image\020\sunglasses\sleepyCombination
230
(913, 25, 45, 1)
(913, 64, 50, 1)
(913, 25, 35, 1)
---
---
---
---
.\Da


1.dataGenerator <br/>
2.Image 減小